In [ ]:
import pandas as pd
import json
import plotly
import plotly.express as px
import datetime as dt
from  geopy.geocoders import Nominatim
import meteomatics.api as weather
from api_config import *
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [ ]:
geolocator = Nominatim(user_agent="HeatML")
loc = geolocator.geocode("Hong Kong"+','+ "China")
lat = loc.latitude
lng = loc.longitude

curr_dt = dt.datetime.utcnow()
prev_dt = curr_dt - dt.timedelta(days=100)
curr_dt = dt.datetime.utcnow() + dt.timedelta(days=1)
curr_dt = str(curr_dt.isoformat()) + "Z"
prev_dt = str(prev_dt.isoformat()) + "Z"
interval = 1 # hours

In [ ]:
# url=f"https://api.meteomatics.com/{prev_dt}--{curr_dt}:PT{interval}H/t_apparent:F/{lat},{lng}/json?model=mix"
url=f"https://api.meteomatics.com/{prev_dt}--{curr_dt}:PT{interval}H/t_apparent:F,soil_moisture_deficit:mm,forest_fire_warning:idx,co:ugm3,o3:ugm3,no2:ugm3,t_1000m:F,t_2m:F/{lat},{lng}/json?model=mix"

In [ ]:
df = weather.query_api(url=url, username=USER, password=PASSWORD)
response = dict(df.json())
response

In [ ]:
data = response["data"]
apparent, soil, fire, co, o3, no2, t1000, t2 = data
apparent, soil, fire, co, o3, no2, t1000, t2 = apparent["coordinates"][0]["dates"], soil["coordinates"][0]["dates"], fire["coordinates"][0]["dates"], co["coordinates"][0]["dates"], o3["coordinates"][0]["dates"], no2["coordinates"][0]["dates"], t1000["coordinates"][0]["dates"], t2["coordinates"][0]["dates"]

In [ ]:
df1 = pd.DataFrame(apparent).rename(columns={"value": "Heat Index (F)", "date": "Datetime"})
df2 = pd.DataFrame(soil).rename(columns={"value": "Soil Moisture (mm)", "date": "Datetime"})
df3 = pd.DataFrame(fire).rename(columns={"value": "Forest Fire Index", "date": "Datetime"})
df4 = pd.DataFrame(co).rename(columns={"value": "Carbon Monoxide (ugm3)", "date": "Datetime"})
df5 = pd.DataFrame(o3).rename(columns={"value": "Ozone (ugm3)", "date": "Datetime"})
df6 = pd.DataFrame(no2).rename(columns={"value": "Nitrogen Dioxide (ugm3)", "date": "Datetime"})
df7 = pd.DataFrame(t1000).rename(columns={"value": "Temp @ 1km (F)", "date": "Datetime"})
df8 = pd.DataFrame(t2).rename(columns={"value": "Temp @ 2m (F)", "date": "Datetime"})
df = df1
df["Soil Moisture (mm)"] = df2["Soil Moisture (mm)"]
df["Forest Fire Index"] = df3["Forest Fire Index"]
df["Carbon Monoxide (ugm3)"] = df4["Carbon Monoxide (ugm3)"]
df["Ozone (ugm3)"] = df5["Ozone (ugm3)"]
df["Nitrogen Dioxide (ugm3)"] = df6["Nitrogen Dioxide (ugm3)"]
df


In [ ]:
fig = px.line(df, x='Datetime', y=["Ozone (ugm3)", "Nitrogen Dioxide (ugm3)"])
fig

In [ ]:
def show_heatmap(df):
    plt.matshow(df.corr())
    plt.xticks(range(df.shape[1]), df.columns, fontsize=14, rotation=90)
    plt.gca().xaxis.tick_bottom()
    plt.yticks(range(df.shape[1]), df.columns, fontsize=14)

    cb = plt.colorbar()
    cb.ax.tick_params(labelsize=14)
    plt.title("Feature Correlation Heatmap", fontsize=14)
    plt.show()

show_heatmap(df)
# df.columns

In [ ]:
from scipy import stats

# df["Datetime"] = pd.to_datetime(df["Datetime"])
# df['date_ordinal'] = pd.to_datetime(df['Datetime']).map(dt.datetime.toordinal)
# slope, intercept, r_value, p_value, std_err = stats.linregress(df['date_ordinal'], df['Heat Index (F)'])
# print(slope,
# intercept,
# r_value,p_value,
# std_err)

In [ ]:
df.index = df['Datetime']
df = df[['Heat Index (F)', 'Soil Moisture (mm)', 'Forest Fire Index',
       'Carbon Monoxide (ugm3)', 'Ozone (ugm3)', 'Nitrogen Dioxide (ugm3)']]
df

In [ ]:
# fig = px.line(df, x='Datetime', y=["Heat Index (F)"])
# fig
df_hi = df["Heat Index (F)"]
df_hi.plot(figsize=(15,10))

In [ ]:
hi_df_mean = df["Heat Index (F)"].rolling(window=30).mean()
hi_df_mean.plot(figsize=(20,15))

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm
result_add = seasonal_decompose(df_hi, model="additive", extrapolate_trend='freq', period=365)

plt.rcParams.update({'figure.figsize': (10,10)})
result_add.plot().suptitle('Additive Decomposition', fontsize=22)
plt.show()

In [ ]:
hi_trend = pd.DataFrame(result_add.trend)["trend"]
hi_trend

In [ ]:
import itertools
p=d=q=range(0,2)
pdq=list(itertools.product(p,d,q))
seasonal_pdq=[(x[0], x[1], x[2], 12) for x in list(itertools.product(p, d, q))]

In [ ]:
import warnings
warnings.filterwarnings("ignore")
max_AIC = 1e20
par = None
seas_par = None

for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = sm.tsa.statespace.SARIMAX(
                df["Heat Index (F)"],
                order=param,
                seasonal_order=param_seasonal,
                enforce_stationarity=False,
                enforce_invertibility=False
            )
            results=mod.fit()
            if results.aic < max_AIC:
                max_AIC = results.aic
                par = param
                seas_par = param_seasonal
        except:
            continue
print(max_AIC, par, seas_par)

In [ ]:
mod = sm.tsa.statespace.SARIMAX(
    df_hi,
    order=par,
    seasonal_order=seas_par,
    enforce_stationarity=False,
    enforce_invertibility=False
)
results = mod.fit()

In [ ]:
results.plot_diagnostics(figsize=(15,12))
plt.show()

In [ ]:
pred = results.forecast(steps=200, dynamic=False, full_results=True)
df_pred = pd.DataFrame(pred)
df_pred

In [ ]:
mn = df_pred["predicted_mean"].mean()
df_pred["Heat Index (F)"] = df_pred["predicted_mean"] + ((df_pred["predicted_mean"] - mn) * 2)


In [ ]:
import plotly.graph_objects as go
# set up plotly figure
df_hi = pd.DataFrame(df_hi)
fig = go.Figure()

# add line / trace 1 to figure
fig.add_trace(go.Line(
    x=df_hi.index,
    y=df_hi['Heat Index (F)'],
    # hovertext=df['A_info'],
    # hoverinfo="text",
    marker=dict(
        color="blue"
    ),
    showlegend=False
))

# add line / trace 2 to figure
fig.add_trace(go.Line(
    x=df_pred.index,
    y=df_pred['Heat Index (F)'],
    # hovertext=df['B_info'],
    # hoverinfo="text",
    marker=dict(
        color="red"
    ),
    showlegend=False
))

fig.show()

In [ ]:
df_all_hi = df_hi.append(df_pred) 
# fig = px.line(df, x='Datetime', y="Forest Fire Index", title="Forest Fire Index Time Series")
df_all_hi = pd.DataFrame(df_all_hi["Heat Index (F)"])
df_all_hi

In [ ]:
df

In [ ]:
from scipy import stats

df_tmp = df
df_tmp["date"] = pd.to_datetime(df_tmp["Datetime"])
df_tmp['date_ordinal'] = pd.to_datetime(df_tmp['Datetime']).map(dt.datetime.toordinal)
slope_hi = stats.linregress(df['date_ordinal'], df["Forest Fire Index"]).slope
last_hi = df["Forest Fire Index"].iat[-1]
last_hi + slope_hi * 100

In [ ]:
fig = px.line(df, x='Datetime', y=["Ozone (ugm3)", "Nitrogen Dioxide (ugm3)", "Carbon Monoxide (ugm3)"])
fig